In [1]:
import numpy as np
import torch
import torch.nn as nn
import torch.optim as optim


/Users/goldenyoo/miniforge3/envs/mac_cpu/lib/python3.8/site-packages/tqdm/auto.py:22: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [2]:
sentences = ["i like dog", "i love coffee", "i hate milk", "you like cat", "you love milk", "you hate coffee"]
dtype = torch.float


In [ ]:
print(sentences)

In [ ]:
print(" ".join(sentences))

In [ ]:
print(" ".join(sentences).split())

In [ ]:
set(" ".join(sentences).split())

In [ ]:
list(set(" ".join(sentences).split()))

In [ ]:
word_list = list(set(" ".join(sentences).split()))
for i, w in enumerate(word_list):
    print(i)
    print(w)


In [3]:
"""
Word Processing
"""
word_list = list(set(" ".join(sentences).split()))
word_dict = {w: i for i, w in enumerate(word_list)}
number_dict = {i: w for i, w in enumerate(word_list)}
n_class = len(word_dict)



In [4]:

"""
TextRNN Parameter
"""
batch_size = len(sentences)
n_step = 2  # 학습 하려고 하는 문장의 길이 - 1
n_hidden = 5  # 은닉층 사이즈

def make_batch(sentences):
  input_batch = []
  target_batch = []

  for sen in sentences:
    word = sen.split()
    input = [word_dict[n] for n in word[:-1]]
    target = word_dict[word[-1]]

    input_batch.append(np.eye(n_class)[input])  # One-Hot Encoding
    target_batch.append(target)
  
  return input_batch, target_batch

input_batch, target_batch = make_batch(sentences)
input_batch = torch.tensor(input_batch, dtype=torch.float32, requires_grad=True)
target_batch = torch.tensor(target_batch, dtype=torch.int64)



/var/folders/ng/3wjwv6195cb09tvny8_v_7_m0000gn/T/ipykernel_44764/626710555.py:23: UserWarning: Creating a tensor from a list of numpy.ndarrays is extremely slow. Please consider converting the list to a single numpy.ndarray with numpy.array() before converting to a tensor. (Triggered internally at  /Users/runner/miniforge3/conda-bld/pytorch-recipe_1647804319176/work/torch/csrc/utils/tensor_new.cpp:210.)
  input_batch = torch.tensor(input_batch, dtype=torch.float32, requires_grad=True)


In [5]:

"""
TextLSTM
"""
class TextLSTM(nn.Module):
  def __init__(self):
    super(TextLSTM, self).__init__()

    self.lstm = nn.LSTM(input_size=n_class, hidden_size=n_hidden, dropout=0.3)
    self.fc = nn.Linear(n_hidden, n_class)

    # self.W = nn.Parameter(torch.randn([n_hidden, n_class]).type(dtype))
    # self.b = nn.Parameter(torch.randn([n_class]).type(dtype))
    # self.Softmax = nn.Softmax(dim=1) # softmax 포함되어있다고 하지 않았나? CrossEntropyLoss에...

  def forward(self, hidden_and_cell, X):
    X = X.transpose(0, 1)
    # outputs, hidden = self.lstm(X, hidden_and_cell)
    # outputs = outputs[-1]  # 최종 예측 Hidden Layer
    # print(X.size())
    outputs, (h_n,c_n) = self.lstm(X, hidden_and_cell)
    
    outputs = h_n[-1]  # 최종 예측 Hidden Layer
    # print(outputs.size())

    # model = torch.mm(outputs, self.W) + self.b  # 최종 예측 최종 출력 층
    model = self.fc(outputs)  # 최종 예측 최종 출력 층
    
    return model
	


In [6]:

"""
Training
"""
model = TextLSTM()
criterion = nn.CrossEntropyLoss()
optimizer = optim.Adam(model.parameters(), lr=0.01)

for epoch in range(500):
  hidden = torch.zeros(1, batch_size, n_hidden, requires_grad=True)
  cell = torch.zeros(1, batch_size, n_hidden, requires_grad=True)
  output = model((hidden, cell), input_batch)
  loss = criterion(output, target_batch)

  if (epoch + 1) % 100 == 0:
    print('Epoch:', '%04d' % (epoch + 1), 'cost =', '{:.6f}'.format(loss))
  
  optimizer.zero_grad()
  loss.backward()
  optimizer.step()

input = [sen.split()[:2] for sen in sentences]

hidden = torch.zeros(1, batch_size, n_hidden, requires_grad=True)
cell = torch.zeros(1, batch_size, n_hidden, requires_grad=True)
predict = model((hidden, cell), input_batch).data.max(1, keepdim=True)[1]
print([sen.split()[:2] for sen in sentences], '->', [number_dict[n.item()] for n in predict.squeeze()])

/Users/goldenyoo/miniforge3/envs/mac_cpu/lib/python3.8/site-packages/torch/nn/modules/rnn.py:62: UserWarning: dropout option adds dropout after all but last recurrent layer, so non-zero dropout expects num_layers greater than 1, but got dropout=0.3 and num_layers=1
  warnings.warn("dropout option adds dropout after all but last "


Epoch: 0100 cost = 0.473329
Epoch: 0200 cost = 0.045445
Epoch: 0300 cost = 0.018823
Epoch: 0400 cost = 0.011123
Epoch: 0500 cost = 0.007530
[['i', 'like'], ['i', 'love'], ['i', 'hate'], ['you', 'like'], ['you', 'love'], ['you', 'hate']] -> ['dog', 'coffee', 'milk', 'cat', 'milk', 'coffee']


In [7]:
list(model.named_parameters())

[('lstm.weight_ih_l0',
  Parameter containing:
  tensor([[ 2.2959e-01,  1.2352e+00,  1.4425e+00,  2.1049e-01, -1.4144e-01,
            9.3685e-01,  6.6125e-02, -1.8781e-01,  1.3220e-01],
          [ 2.0392e-01,  1.4550e-01,  1.2286e+00,  4.5497e-01, -1.0107e-01,
            1.0979e+00,  8.3179e-01, -1.3570e-01,  4.0171e-01],
          [ 2.0629e-01,  6.7606e-01,  1.5959e+00,  4.3274e-01, -5.5717e-02,
            3.7154e-01,  1.0130e+00,  1.6352e-01, -1.9693e-01],
          [-3.4550e-01, -1.8735e-01,  7.6785e-01,  9.4921e-01, -4.0330e-01,
            8.1932e-01,  7.9812e-01, -4.2112e-01,  1.5220e-01],
          [-2.9098e-01,  1.1089e+00,  1.8268e+00, -2.1308e+00, -3.3672e-01,
            1.5863e+00,  6.4966e-01, -3.4723e-01, -3.3569e-01],
          [ 2.9612e-02,  5.6132e-01, -1.7194e-01, -1.5424e+00,  1.9196e-01,
            2.3765e-01,  3.3198e+00, -3.3491e-01,  1.2823e-01],
          [-3.7450e-01, -1.4594e+00,  4.2409e-01,  1.8632e+00,  1.1670e-01,
           -3.5461e-01,  2.4244e+00, 